DHIA SALSABILA AZHAR

18/430187/PA/18700

### Pendahuluan

[Keras](https://github.com/keras-team/keras/blob/master/README.md) memiliki banyak model *deep learning* untuk klasifikasi gambar yang siap pakai. Dengan mengatur parameter ``weights='imagenet'``, *weight* hasil *training* berhari-hari pada jutaan gambar dari dataset ImageNet otomatis di*download*. Tanpa kita *training* di PC sendiri, model dapat kita masukkan gambar sembarang untuk diklasifikasi.

Jumlah kelas dataset ImageNet sangat banyak, ada 1000. Perhatikan sampel berikut. 

>#### ImageNet [classes.txt](https://github.com/xmartlabs/caffeflow/blob/master/examples/imagenet/imagenet-classes.txt)
>... head cabbage broccoli cauliflower zucchini, courgette spaghetti squash acorn squash butternut squash cucumber, cuke artichoke, globe artichoke bell pepper cardoon mushroom Granny Smith strawberry orange lemon fig pineapple, ananas **banana** jackfruit, jak, jack custard apple pomegranate ...

Model mampu membedakan berbagai sayur dan buah. Akan tetapi, model tidak cukup spesifik untuk membedakan varian dari setiap sayur atau buah dalam perbendaharaannya. Model akan memprediksi, misalnya, 'orange' ketika dimasukkan gambar jeruk mandarin maupun jeruk sunkist. Pada artikel ini kita akan membuat model yang khusus membedakan antara **pisang tanduk** dan **pisang kepok**. Kita akan menggunakan salah satu model yang disediakan Keras, DenseNet121.

### DenseNet

DenseNet models {121, 169, 201}, with weights pre-trained on ImageNet.

This model and can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).

The default input size for this model is 224x224.

```
keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

```

#### Arguments
+ blocks: numbers of building blocks for the four dense layers.
+ include_top: whether to include the fully-connected layer at the top of the network.
+ weights: one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
+ input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
+ input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.
+ pooling: optional pooling mode for feature extraction when include_top is False. None means that the output of the model will be the 4D tensor output of the last convolutional layer. avg means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor. max means that global max pooling will be applied.
+ classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.


### Imports

Import secukupnya :)

In [ ]:
from keras.applications import densenet
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import matplotlib.pyplot as plt

### DataSet

Gunakan dataset Magnetic Tiles dataset pada Praktikum 4 : Inspeksi Cacat Bahan (Magnetic Tiles Defect).

Simpan gambar-gambar pisang sehingga struktur folder kita menjadi seperti berikut.

    image_train/
        MT_Blowhole/
            ...
        MT_Break/
            ...
        MT_Crack/
            ...
        MT_Fray/
            ...   
        MT_Free/
            ...       
    image_test/
        MTBLW001.jpg
        MTBLW002.jpg
        ...
        MTCRK018.jpg
        MTFREE001.jpg
        MTFREE002.jpg
        ...
        MTUNEVEN018.jpg

Catatan: penamaan data *train* bebas, sedangkan nama file data *test* sebaiknya mengandung kelasnya untuk mempermudah membaca hasil prediksi nanti.

Tempatkan dataset pada Google Colabs kalian sebagai contoh di dalam folder : Colab Notebooks/dataset tiles defect

In [ ]:
from google.colab import drive

# mounting dataset from gdrive
drive.mount('/content/gdrive', force_remount=True)

# dataset path
root_path = '/content/gdrive/My Drive/dataset tiles defect/dataset tiles defect'

print("Path root:", root_path)

Mounted at /content/gdrive
Path root: /content/gdrive/My Drive/dataset tiles defect/dataset tiles defect


### READ DATASET


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(root_path + '/image_train', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    batch_size=32, 
                                                    class_mode='categorical', 
                                                    shuffle=True)
category_dict = train_generator.class_indices
print(category_dict)

Found 1254 images belonging to 6 classes.
{'MT_Blowhole': 0, 'MT_Break': 1, 'MT_Crack': 2, 'MT_Fray': 3, 'MT_Free': 4, 'MT_Uneven': 5}


In [ ]:
number_of_classes = len(category_dict)

default_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

x = default_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds_ = Dense(number_of_classes, activation='softmax')(x)

def_model = Model(inputs=default_model.input, outputs=preds_)

n_freeze = 300
for layer in model.layers[:n_freeze]:
    layer.trainable=False
for layer in model.layers[n_freeze:]:
    layer.trainable=True

print(len(model.layers))

58892288/58889256 [==============================] - 1s 0us/step


Catatan: Jumlah layer dapat diketahui dengan ```print(len(model.layers))```. 

Perintah terakhir mengatur supaya 300 layer pertama di*freeze*, yakni hanya layer ke 301 sampai layer sebelum *output* saja yang di*train* parameternya. Jika mau, kita juga dapat mengatur supaya hanya layer *output* saja yang di*train* (murni *transfer learning*) seperti berikut.

    for layer in model.layers:
        layer.trainable=False

```decode_predictions``` dapat digunakan untuk membuat keluaran model (*array* 1000 kelas) menjadi *human-readable*.
```
prediction = model.predict(x)
keras.applications.densenet.decode_predictions(prediction, top=2)
```
Kebutuhan pengujian model kita dapat diimplementasi dengan *get key by value in dictionary*. (Saya kesulitan mencari *method* serupa yang lebih generik, yakni dapat digunakan untuk *transfer learning* dengan jumlah kelas ≠ 1000.)

In [ ]:
test_path = root_path+'/image_test/'

for directory in os.listdir(test_path):
    
    # Load image.
    img_path = test_path+directory
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = def_model.predict(x)
    default_preds = def_model.predict(x)

    # Printing results.

    # Default 1000 classes (without transfer learning).

    # Print transfer learning model top-1
    confidence_array = preds[0]
    index_max = np.argmax(confidence_array)

    confidence_array_notrf = default_preds[0]
    index_max_notrf = np.argmax(confidence_array_notrf)

    # Get KEY (category) by VALUE (index_max) in dictionary
    # mydict = {'george':16,'amber':19}
    # print(list(mydict.keys())[list(mydict.values()).index(16)]) # Example in one line.

    category_names = category_dict.keys()
    category_values = category_dict.values()
    category_at_index = list(category_values).index(index_max_notrf)
    category_max = list(category_names)[category_at_index]
    
    print(f"Without Transfer Learning Top-2 [{directory}]: \nTop-1 (confidence)\n{category_max} ({max(confidence_array)*100}%)")

    category_at_index = list(category_values).index(index_max)
    category_max = list(category_names)[category_at_index]

    print(f"\nWith Transfer Learning [{directory}]: \nTop-1 (confidence)\n{category_max} ({max(confidence_array)*100}%)")

    # Print transfer learning model all classes
    print("\nClass (confidence)")

    for category in category_dict:
        category_index = category_dict[category]
        value = confidence_array[category_index] * 100
        print(f"{category} ({value}%)")

    print("\n============================\n")


Without Transfer Learning Top-2 [MTFRY002.jpg]: 
Top-1 (confidence)
MT_Uneven (18.681561946868896%)

With Transfer Learning [MTFRY002.jpg]: 
Top-1 (confidence)
MT_Uneven (18.681561946868896%)

Class (confidence)
MT_Blowhole (17.698021233081818%)
MT_Break (14.204610884189606%)
MT_Crack (13.368022441864014%)
MT_Fray (17.8570494055748%)
MT_Free (18.190734088420868%)
MT_Uneven (18.681561946868896%)


Without Transfer Learning Top-2 [MTUNEVEN002.jpg]: 
Top-1 (confidence)
MT_Uneven (18.85906010866165%)

With Transfer Learning [MTUNEVEN002.jpg]: 
Top-1 (confidence)
MT_Uneven (18.85906010866165%)

Class (confidence)
MT_Blowhole (16.35918766260147%)
MT_Break (14.300143718719482%)
MT_Crack (13.796254992485046%)
MT_Fray (17.972087860107422%)
MT_Free (18.71325969696045%)
MT_Uneven (18.85906010866165%)


Without Transfer Learning Top-2 [MTBLW007.jpg]: 
Top-1 (confidence)
MT_Fray (20.334358513355255%)

With Transfer Learning [MTBLW007.jpg]: 
Top-1 (confidence)
MT_Fray (20.334358513355255%)

Class (c

Catatan: Hasil yang lebih dulu di*print* adalah yang lebih dulu terbaca oleh ```os.listdir()```.

In [ ]:
for directory in os.listdir(test_path):
    print(directory, end=' ')

MTFRY002.jpg MTUNEVEN002.jpg MTBLW007.jpg MTUNEVEN001.jpg MTUNEVEN016.jpg MTUNEVEN022.jpg MTBRK013.jpg MTFREE019.jpg MTCRK001.jpg MTBLW001.jpg MTBRK010.jpg MTFREE028.jpg MTCRK011.jpg MTFREE004.jpg MTBRK018.jpg MTFREE013.jpg MTBRK009.jpg MTFRY001.jpg MTBRK007.jpg MTUNEVEN021.jpg MTBLW003.jpg MTFREE017.jpg MTFREE021.jpg MTBLW005.jpg MTBRK005.jpg MTUNEVEN006.jpg MTBLW012.jpg MTFREE022.jpg MTFRY009.jpg MTBRK017.jpg MTUNEVEN007.jpg MTUNEVEN010.jpg MTFRY004.jpg MTFREE008.jpg MTFREE020.jpg MTBRK004.jpg MTCRK008.jpg MTBLW006.jpg MTFREE005.jpg MTFREE026.jpg MTBRK016.jpg MTBRK011.jpg MTUNEVEN008.jpg MTCRK016.jpg MTCRK010.jpg MTFREE025.jpg MTFREE010.jpg MTCRK017.jpg MTBRK012.jpg MTUNEVEN003.jpg MTUNEVEN012.jpg MTFREE003.jpg MTCRK005.jpg MTBLW010.jpg MTBLW016.jpg MTCRK014.jpg MTBLW015.jpg MTFREE023.jpg MTBRK014.jpg MTUNEVEN017.jpg MTFRY008.jpg MTCRK009.jpg MTBLW014.jpg MTUNEVEN020.jpg MTFRY003.jpg MTFRY006.jpg MTFREE006.jpg MTBLW009.jpg MTCRK013.jpg MTCRK003.jpg MTBRK019.jpg MTUNEVEN015.jpg MTBRK0

### Selesai!

Sesuai motivasi dibuatnya Keras sendiri "*easy and fast prototyping*", notebook ini dibuat generik. 

Sumber : https://github.com/tbmreza/comvis-notes/blob/master/transfer-learning.ipynb

dengan beberapa penyesuaian